# Predikcia vsetkych parametrov pomocou jednej NN
## Normovane overcontact data

In [1]:
# Blok 1 - nacitanie kniznic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from keras.utils import np_utils
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Input, Dense, LSTM, Dropout, Flatten
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

np.random.seed(1234)
pd.set_option('display.max_rows', None)

In [2]:
# Blok 2 - Funkcie pre generovanie sumu. Nastavenie pseudo-nahodneho generatora.

def generate_observation_sigma(space_obs_frac=0.5):
    """
    Draws a standard deviation of noise in light curve points from a "true" value provided in synthetic light curve.
    Noise sigma is drawn from bimodal distribution taking into account contributions from space based and earth based
    observations which have different levels of stochastic noise.

    :param space_obs_frac: ratio between earth based and space based observations
    :return: float; standard deviation of the light curve noise
    """
    earth_based_sigma = 4e-3
    space_based_sigma = 2e-4
    sigma = np.random.choice([earth_based_sigma, space_based_sigma], p=[1-space_obs_frac, space_obs_frac])
    return np.random.rayleigh(sigma)

def stochastic_noise_generator(curve):
    """
    Introduces gaussian noise into synthetic observation provided in `curve`.

    :param curve: numpy.array; normalized light curve
    :return: Tuple(numpy.array, float); normalized light curve with added noise, standard deviation of observations
    """
    sigma = generate_observation_sigma()
    return np.random.normal(curve, sigma), np.full(curve.shape, sigma)

## Data loading

In [3]:
# Blok 3 - nacitanie dat
data = pd.read_pickle("overcontact_all_parameters.pkl").reset_index()

In [4]:
# Blok 4 - vyselektovanie nahodnej vzorky dat o velkosti 100 000
data_sample = data.sample(n=100000)

## Train test split

In [5]:
# Blok 5 - vytvorenie pola kriviek
X = []
for row in data_sample["curve"]:
    X.append(row)
X=np.array(X)

In [6]:
# Blok 6 - vytvorenie pola features, ktore bude model predikovat
y = np.array(data_sample[[
    "primary__t_eff",
    "secondary__t_eff",
    "inclination",
    "mass_ratio",
    "primary__surface_potential",
    "secondary__surface_potential",
    "t1/t2",
    "critical_surface_potential",
    "primary__equivalent_radius",
    "secondary__equivalent_radius",
    "primary__filling_factor",
    "secondary__filling_factor"]])

In [7]:
# Blok 7 - zadefinovanie scalera pre min max normalizaciu dat
scaler = MinMaxScaler()
y_minmax_scaled = scaler.fit_transform(y)
y_minmax_scaled[0]

array([0.8       , 0.8       , 0.79148929, 0.49494949, 0.53031622,
       0.53031622, 0.        , 0.54862702, 0.1671155 , 0.85762583,
       0.48776465, 0.48776465])

In [8]:
# Blok 8 - rozdelenie dat na trenovaciua a testovaciu mnozinu v pomere 80/20
X_train1, X_test, y_train1, y_test = train_test_split(X, y_minmax_scaled, test_size=0.2)

In [9]:
# Blok 9 - pridanie sumu do trenovacich dat
X_train_n = []
y_train_n = []
for i in range(len(X_train1)):
    for j in range(3):
        curve = stochastic_noise_generator(X_train1[i])
        X_train_n.append(curve[0])
        y_train_n.append(y_train1[i])
X_train_n = np.array(X_train_n)
y_train_n=np.array(y_train_n)

In [15]:
# Blok 10 - vypis poctu dat v jednotlivych datovych mnozinach
print("Number of records in dataset: ", len(data),
    "\nNumber of records in sample: ", len(X),
    "\nNumber of train data without noise: ", len(X_train1),
    "\nNumber of train data with noise: ", len(X_train_n),
    "\nNumber of test data without noise: ", len(X_test))

Number of records in dataset:  1212796 
Number of records in sample:  100000 
Number of train data without noise:  80000 
Number of train data with noise:  240000 
Number of test data without noise:  20000


## Model

In [16]:
# Blok 11 - zadefinovanie architektury modelu NN
inputs = Input(shape=(400, 1))
b = Conv1D(64, kernel_size = 3, padding = "valid")(inputs)
b = MaxPooling1D(2)(b)
b = Dropout(0.2)(b)
b = LSTM(64, return_sequences=True)(b)
b = Flatten()(b)
b = Dense(64, activation='relu')(b)
x = Dense(32, activation='relu')(b)
output = Dense(12, activation='linear')(x)
model = Model(inputs=inputs, outputs=output)
model.compile(loss='mse', optimizer='adam', metrics=["mae", "mape"])

saved_model = "models/norm_overcontact_all_params.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor = 'val_mae', verbose = 1, save_best_only = True, mode = 'min')
early = EarlyStopping(monitor = "val_mae", mode = "min", patience = 25)
callbacks_list = [checkpoint, early]

print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 398, 64)           256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 199, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 199, 64)           33024     
_________________________________________________________________
flatten (Flatten)            (None, 12736)             0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [18]:
# Blok 12 - trenovanie modelu
history = model.fit(X_train_n, y_train_n, validation_split = 0.1, epochs = 10, verbose = 1, callbacks = callbacks_list, batch_size = 64)

Epoch 1/10
3375/3375 [==============================] - ETA: 0s - loss: 0.0285 - mae: 0.1177 - mape: 6007802.0000
Epoch 00001: val_mae improved from inf to 0.08945, saving model to models\norm_overcontact_all_params.hdf5
3375/3375 [==============================] - 398s 118ms/step - loss: 0.0285 - mae: 0.1177 - mape: 6007802.0000 - val_loss: 0.0182 - val_mae: 0.0894 - val_mape: 4261444.0000
Epoch 2/10
3375/3375 [==============================] - ETA: 0s - loss: 0.0181 - mae: 0.0895 - mape: 3811119.0000
Epoch 00002: val_mae improved from 0.08945 to 0.08291, saving model to models\norm_overcontact_all_params.hdf5
3375/3375 [==============================] - 403s 120ms/step - loss: 0.0181 - mae: 0.0895 - mape: 3811119.0000 - val_loss: 0.0160 - val_mae: 0.0829 - val_mape: 3664032.7500
Epoch 3/10
3375/3375 [==============================] - ETA: 0s - loss: 0.0167 - mae: 0.0841 - mape: 3351118.7500
Epoch 00003: val_mae improved from 0.08291 to 0.07869, saving model to models\norm_overcontact

In [10]:
# Blok 13 - necitanie modelu
model = load_model("models/norm_overcontact_all_params.hdf5")

## Model evaluation on normalized test data

In [11]:
# Blok 14 - evaluacia modelu na testovacej mnozine bez sumu
scores = model.evaluate(X_test, y_test)
print('Loss: {:.4f}, MAE: {:.4f}'.format(scores[0], scores[1]))

625/625 [==============================] - 31s 50ms/step - loss: 0.0123 - mean_absolute_error: 0.0656 - mean_absolute_percentage_error: 2601702.5000
Loss: 0.0123, MAE: 0.0656


In [12]:
# Blok 15 - pridanie sumu do testovacich dat
X_test_n = []
y_test_norm_n = []
for i in range(len(X_test)):
    for j in range(3):
        curve = stochastic_noise_generator(X_test[i])
        X_test_n.append(curve[0])
        y_test_norm_n.append(y_test[i])
        j += 1
X_test_n = np.array(X_test_n)
y_test_norm_n = np.array(y_test_norm_n)

In [13]:
# Blok 16 - evaluacia modelu na testovacej mnozine so sumom
scores_n = model.evaluate(X_test_n, y_test_norm_n)
print('Loss: {:.4f}, MAE: {:.4f}'.format(scores_n[0], scores_n[1]))

1875/1875 [==============================] - 141s 75ms/step - loss: 0.0126 - mean_absolute_error: 0.0674 - mean_absolute_percentage_error: 2633009.0000
Loss: 0.0126, MAE: 0.0674


### Prediction on normalized test data without noise + inverse normalization

In [14]:
# Blok 17 - predikcia na datach bez sumu
y_pred_norm = model.predict(X_test)

In [15]:
# Blok 18 - spatna normalizacia predikcii
denorm = scaler.inverse_transform(y_pred_norm)
denorm[0]

array([6.9901250e+03, 6.6553823e+03, 9.4541442e-01, 3.9031935e+00,
       7.5019617e+00, 7.5348935e+00, 1.0541420e+00, 7.7449927e+00,
       2.8597540e-01, 5.1029235e-01, 2.9373312e-01, 2.9713902e-01],
      dtype=float32)

In [16]:
# Blok 19 - vytvorenie df z denormovanych predikcii
denorm_pred_df = pd.DataFrame(denorm,
                           columns = [
                                "P_prim__t_eff",
                                "P_sec__t_eff",
                                "P_inclination",
                                "P_mass_ratio",
                                "P_prim__surface_potential",
                                "P_sec__surface_potential",
                                "P_t1_t2",
                                "P_critical_surface_potential",
                                "P_primary_equivalent_radius",
                                "P_secondary_equivalent_radius",
                                "P_primary_filling_factor",
                                "P_secondary_filling_factor"
                            ])
denorm_pred_df.head()

,P_prim__t_eff,P_sec__t_eff,P_inclination,P_mass_ratio,P_prim__surface_potential,P_sec__surface_potential,P_t1_t2,P_critical_surface_potential,P_primary_equivalent_radius,P_secondary_equivalent_radius,P_primary_filling_factor,P_secondary_filling_factor
0,6990.125000,6655.382324,0.945414,3.903193,7.501962,7.534894,1.054142,7.744993,0.285975,0.510292,0.293733,0.297139
1,6752.203125,6624.109375,0.915045,3.697112,7.150830,7.194067,1.025620,7.526182,0.306539,0.531069,0.558212,0.563122
2,6846.311523,6500.617676,1.522212,1.108788,3.781532,3.790986,1.056917,3.936830,0.396615,0.415668,0.293223,0.299829
3,5917.981934,5570.070312,0.915215,1.313440,3.850923,3.905050,1.067454,4.273981,0.424953,0.474609,0.723804,0.726519
4,6731.123535,6471.833496,1.496251,0.046823,2.107799,2.306593,1.045047,2.082450,0.519066,0.237436,0.250759,0.235542


In [17]:
# Blok 20 - vypocet priemernych hodnot
pred_mean = denorm_pred_df.mean(axis=0)
pred_mean

P_prim__t_eff                    6430.596191
P_sec__t_eff                     6149.156250
P_inclination                       1.229885
P_mass_ratio                        1.405825
P_prim__surface_potential           3.968041
P_sec__surface_potential            4.008068
P_t1_t2                             1.048048
P_critical_surface_potential        4.318586
P_primary_equivalent_radius         0.427423
P_secondary_equivalent_radius       0.452089
P_primary_filling_factor            0.631660
P_secondary_filling_factor          0.630456
dtype: float32

In [22]:
# Blok 21 - vytvorenie df z testovaich dat bez sumu, vypocet priemernych hodnot
denorm_test = scaler.inverse_transform(y_test)
y_test_norm = pd.DataFrame(denorm_test,
                            columns = [
                                "P_prim__t_eff",
                                "P_sec__t_eff",
                                "P_inclination",
                                "P_mass_ratio",
                                "P_prim__surface_potential",
                                "P_sec__surface_potential",
                                "P_t1_t2",
                                "P_critical_surface_potential",
                                "P_primary_equivalent_radius",
                                "P_secondary_equivalent_radius",
                                "P_primary_filling_factor",
                                "P_secondary_filling_factor"
                            ])
true_mean = y_test_norm.mean(axis=0)
true_mean

P_prim__t_eff                    6542.375000
P_sec__t_eff                     6252.187500
P_inclination                       1.221348
P_mass_ratio                        1.412813
P_prim__surface_potential           3.972770
P_sec__surface_potential            3.972770
P_t1_t2                             1.048076
P_critical_surface_potential        4.286547
P_primary_equivalent_radius         0.431312
P_secondary_equivalent_radius       0.446961
P_primary_filling_factor            0.615457
P_secondary_filling_factor          0.615457
dtype: float64

In [23]:
# Blok 22 - vytvorenie df na porovnanie priemernych predikovanych a skutocnych hodnot
eval_pred = pd.DataFrame({'attribute': true_mean.index,
            'avg_true': true_mean.values,
            'avg_pred': pred_mean.values,
            'MAE': abs(true_mean.values - pred_mean.values)})
eval_pred

,attribute,avg_true,avg_pred,MAE
0,P_prim__t_eff,6542.375000,6430.596191,111.778809
1,P_sec__t_eff,6252.187500,6149.156250,103.031250
2,P_inclination,1.221348,1.229885,0.008536
3,P_mass_ratio,1.412813,1.405825,0.006989
4,P_prim__surface_potential,3.972770,3.968041,0.004728
5,P_sec__surface_potential,3.972770,4.008068,0.035298
6,P_t1_t2,1.048076,1.048048,0.000028
7,P_critical_surface_potential,4.286547,4.318586,0.032040
8,P_primary_equivalent_radius,0.431312,0.427423,0.003889
9,P_secondary_equivalent_radius,0.446961,0.452089,0.005128


## Prediction on normalized test data with noise

In [24]:
# Blok 23 - predikcia na datach so sumom
y_pred_norm_n = model.predict(X_test_n)

In [25]:
# Blok 24 - spatna normalizacia predikcii
denorm_n = scaler.inverse_transform(y_pred_norm)
denorm_n[0]

array([6.9901250e+03, 6.6553823e+03, 9.4541442e-01, 3.9031935e+00,
       7.5019617e+00, 7.5348935e+00, 1.0541420e+00, 7.7449927e+00,
       2.8597540e-01, 5.1029235e-01, 2.9373312e-01, 2.9713902e-01],
      dtype=float32)

In [26]:
# Blok 25 - vytvorenie df z denormovanych predikcii
denorm_pred_n_df = pd.DataFrame(denorm_n,
                           columns = [
                                "P_prim__t_eff",
                                "P_sec__t_eff",
                                "P_inclination",
                                "P_mass_ratio",
                                "P_prim__surface_potential",
                                "P_sec__surface_potential",
                                "P_t1_t2",
                                "P_critical_surface_potential",
                                "P_primary_equivalent_radius",
                                "P_secondary_equivalent_radius",
                                "P_primary_filling_factor",
                                "P_secondary_filling_factor"
                            ])
denorm_pred_n_df.head()

,P_prim__t_eff,P_sec__t_eff,P_inclination,P_mass_ratio,P_prim__surface_potential,P_sec__surface_potential,P_t1_t2,P_critical_surface_potential,P_primary_equivalent_radius,P_secondary_equivalent_radius,P_primary_filling_factor,P_secondary_filling_factor
0,6990.125000,6655.382324,0.945414,3.903193,7.501962,7.534894,1.054142,7.744993,0.285975,0.510292,0.293733,0.297139
1,6752.203125,6624.109375,0.915045,3.697112,7.150830,7.194067,1.025620,7.526182,0.306539,0.531069,0.558212,0.563122
2,6846.311523,6500.617676,1.522212,1.108788,3.781532,3.790986,1.056917,3.936830,0.396615,0.415668,0.293223,0.299829
3,5917.981934,5570.070312,0.915215,1.313440,3.850923,3.905050,1.067454,4.273981,0.424953,0.474609,0.723804,0.726519
4,6731.123535,6471.833496,1.496251,0.046823,2.107799,2.306593,1.045047,2.082450,0.519066,0.237436,0.250759,0.235542


In [27]:
# Blok 26 - vypocet priemernych hodnot
pred_mean_n = denorm_pred_n_df.mean(axis=0)
pred_mean_n

P_prim__t_eff                    6430.596191
P_sec__t_eff                     6149.156250
P_inclination                       1.229885
P_mass_ratio                        1.405825
P_prim__surface_potential           3.968041
P_sec__surface_potential            4.008068
P_t1_t2                             1.048048
P_critical_surface_potential        4.318586
P_primary_equivalent_radius         0.427423
P_secondary_equivalent_radius       0.452089
P_primary_filling_factor            0.631660
P_secondary_filling_factor          0.630456
dtype: float32

In [31]:
# Blok 27 - vytvorenie df z testovaich dat bez sumu, vypocet priemernych hodnot
denorm_test_n = scaler.inverse_transform(y_test_norm_n)
y_test_norm_n = pd.DataFrame(denorm_test_n,
                            columns = [
                                "P_prim__t_eff",
                                "P_sec__t_eff",
                                "P_inclination",
                                "P_mass_ratio",
                                "P_prim__surface_potential",
                                "P_sec__surface_potential",
                                "P_t1_t2",
                                "P_critical_surface_potential",
                                "P_primary_equivalent_radius",
                                "P_secondary_equivalent_radius",
                                "P_primary_filling_factor",
                                "P_secondary_filling_factor"
                            ])
true_mean_n = y_test_norm_n.mean(axis=0)
true_mean_n

P_prim__t_eff                    6542.375000
P_sec__t_eff                     6252.187500
P_inclination                       1.221348
P_mass_ratio                        1.412813
P_prim__surface_potential           3.972770
P_sec__surface_potential            3.972770
P_t1_t2                             1.048076
P_critical_surface_potential        4.286547
P_primary_equivalent_radius         0.431312
P_secondary_equivalent_radius       0.446961
P_primary_filling_factor            0.615457
P_secondary_filling_factor          0.615457
dtype: float64

In [32]:
# Blok 28 - vytvorenie df na porovnanie priemernych predikovanych a skutocnych hodnot
eval_pred = pd.DataFrame({'attribute': true_mean_n.index,
            'avg_true': true_mean_n.values,
            'avg_pred': pred_mean_n.values,
            'MAE': abs(true_mean_n.values - pred_mean_n.values)})
eval_pred

,attribute,avg_true,avg_pred,MAE
0,P_prim__t_eff,6542.375000,6430.596191,111.778809
1,P_sec__t_eff,6252.187500,6149.156250,103.031250
2,P_inclination,1.221348,1.229885,0.008536
3,P_mass_ratio,1.412813,1.405825,0.006989
4,P_prim__surface_potential,3.972770,3.968041,0.004728
5,P_sec__surface_potential,3.972770,4.008068,0.035298
6,P_t1_t2,1.048076,1.048048,0.000028
7,P_critical_surface_potential,4.286547,4.318586,0.032040
8,P_primary_equivalent_radius,0.431312,0.427423,0.003889
9,P_secondary_equivalent_radius,0.446961,0.452089,0.005128
